In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ray
#ray.init(huge_pages=True, plasma_directory="/mnt/hugepages")
ray.init()
@ray.remote(num_cpus=1,num_gpus=0)
class ParameterServer():
	def __init__(self):
		self.parameters = dict()
		self.parameters['q'] = dict()

	def get(self):
		return self.parameters

	def put_Q_dict(self, new_q_dict):
		self.parameters['q'] = new_q_dict

In [ ]:
from actor import Actor
from learner import Learner
from replay import ReplayMemory

num_actors = 3
B = 50
periodic_update_frequency = 400
shared_experience_replay_size = 2 * 10 ** 6
num_actions = 6
env = 'PongNoFrameskip-v4'

shared_memory = ReplayMemory.remote(shared_experience_replay_size, alpha=0.6)
shared_state = ParameterServer.remote()

learner = Learner.remote(num_actions, shared_memory = shared_memory, shared_state = shared_state)

actors = [Actor.remote(i, env, shared_memory, shared_state, n=3) for i in range(num_actors)]

In [ ]:
import time
time.sleep(5)

In [ ]:
[a.run.remote(1000) for a in actors]#actors[0].run.remote(1000)

In [ ]:
learner.learn.remote(10000)

In [ ]:
experience_batch = ray.get(shared_memory.sample.remote(32, 0.6))
#new_priorities, loss = self.train(experience_batch)

In [ ]:
device = 'cuda'
s,a,r,s_prime, gamma, qS_tpn, qS_t, key, weights, idxes = experience_batch
#s,a,r,s_prime,done_mask,weights,idxes = experience_batch

weights = torch.Tensor(weights).to(device)
s = torch.as_tensor(s).to(device)
a = torch.LongTensor(a).unsqueeze(1).to(device)
r = torch.as_tensor(r).unsqueeze(1).to(device)
s_prime = torch.as_tensor(s_prime).to(device)
gamma = torch.Tensor(gamma).unsqueeze(1).to(device)

In [ ]:
from models import Qnet
q = Qnet(84,84,4,6).to(device)
q_target = Qnet(84,84,4,6).to(device)

In [ ]:
q_out = q(s)
q_a = q_out.gather(1,a)

argmax_q = q(s_prime).argmax(1).unsqueeze(1)
q_prime = q_target(s_prime).gather(1,argmax_q)
target = r + gamma * q_prime# * done_mask
TD_errors = q_a - target
with torch.no_grad():
    new_priorities = np.abs(TD_errors.cpu()) + 10 ** -5
loss = ((TD_errors ** 2).view(-1) * weights).mean()
print(loss)

In [ ]:
ray.get(shared_memory.sample.remote(2,0.5))
ray.get(shared_memory.size.remote())

In [ ]:
import torch
import numpy as np

In [ ]:
from models import Qnet
net = Qnet(84,84,4,6)
net2 = Qnet(84,84,4,6).cuda()
data = torch.rand((1,4,84,84))
data2 = data.cuda()

In [ ]:
%%time
with torch.no_grad():
    out = net(data).numpy()

In [ ]:
%%time
with torch.no_grad():
    out = net2(data2).cpu().numpy()